In [16]:
import json
import string
from typing import List
from dataclasses import dataclass
import uuid

import openai
from autogen_core import (
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    DefaultTopicId,
    FunctionCall,
    Image,
    MessageContext,
    TypeSubscription,
    message_handler,
    type_subscription,
)
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage, AssistantMessage, LLMMessage
from autogen_core.tools import  FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

from IPython.display import display, Markdown
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
class GroupChatMessage(BaseModel):
    body: UserMessage

class RequestToSpeak(BaseModel):
    pass

# Base

In [18]:
class BaseGroupChatAgent(RoutedAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient, system_message: str, topic_id: TopicId = DefaultTopicId("group_chat")):
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type
        self._topic_id = topic_id
        self._model_client = model_client
        self._system_message = SystemMessage(content=system_message)
        self._chat_history: List[LLMMessage] = []

    @message_handler
    async def handle_message(self, message: GroupChatMessage, context: MessageContext) -> None:
        self._chat_history.extend([
            UserMessage(content=f"Transferred to {message.body.source}", source='system'),
            message.body,
        ])

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, context: MessageContext) -> None:
        display(Markdown(f"### {self.id.type}"))
        self._chat_history.append(UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system"))

        completion = await self._model_client.create(messages=[self._system_message] + self._chat_history)
        self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))
        assert isinstance(completion.content, str), "Completion content must be a string"

        display(Markdown(f"#### {self.id.type} Response"))
        display(Markdown(completion.content))

        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=completion.content, source=self.id.type)),
            topic_id=TopicId(self._group_chat_topic_type, self.id.type)
        )

# Editor and Writer

In [19]:
class WriterAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient):
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are a Writer. You produce good work.",
        )

class EditorAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient):
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are an Editor. Plan and guide the task given by the user. Provide critical feedbacks to the draft and illustration produced by Writer and Illustrator. "
            "Approve if the task is completed and the draft and illustration meets user's requirements.",
        )

# Illustraror agent with image generation capabilities

In [20]:
class IllustratorAgent(BaseGroupChatAgent):
    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
        image_client: openai.AsyncClient,
    ) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are an Illustrator. You use the generate_image tool to create images given user's requirement. "
            "Make sure the images have consistent characters and style.",
        )
        self._image_client = image_client
        self._image_gen_tool = FunctionTool(
            self._image_gen, name="generate_image", description="Call this to generate an image. "
        )

    async def _image_gen(
        self, character_appearence: str, style_attributes: str, worn_and_carried: str, scenario: str
    ) -> str:
        prompt = f"Digital painting of a {character_appearence} character with {style_attributes}. Wearing {worn_and_carried}, {scenario}."
        response = await self._image_client.images.generate(
            prompt=prompt, model="dall-e-3", response_format="b64_json", size="1024x1024"
        )
        return response.data[0].b64_json  # type: ignore

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:  # type: ignore
        display(Markdown(f"### {self.id.type}: "))
        self._chat_history.append(
            UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system")
        )
        # Ensure that the image generation tool is used.
        completion = await self._model_client.create(
            [self._system_message] + self._chat_history,
            tools=[self._image_gen_tool],
            extra_create_args={"tool_choice": "required"},
            cancellation_token=ctx.cancellation_token,
        )
        assert isinstance(completion.content, list) and all(
            isinstance(item, FunctionCall) for item in completion.content
        )
        images: List[str | Image] = []
        for tool_call in completion.content:
            arguments = json.loads(tool_call.arguments)
            display(arguments)
            result = await self._image_gen_tool.run_json(arguments, ctx.cancellation_token)
            image = Image.from_base64(self._image_gen_tool.return_value_as_string(result))
            image = Image.from_pil(image.image.resize((256, 256)))
            display(image.image)  # type: ignore
            images.append(image)
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=images, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )

# User Agent

In [21]:
class UserAgent(RoutedAgent):
    def __init__(self, description: str, group_chat_topic_type: str) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type

    @message_handler
    async def handle_message(self, message: GroupChatMessage, context: MessageContext) -> None:
        display(Markdown(f"### {self.id.type}"))
        ## For frontend

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, context: MessageContext) -> None:

        user_input = input("Enter your message, type 'APPROVE' to approve the task, or 'REJECT' to reject it: ")
        display(Markdown(f"### {self.id.type}"))
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=user_input, source=self.id.type)),
            topic_id=DefaultTopicId(self._group_chat_topic_type)
        )

#  Group Chat Manager

In [22]:
class GroupChatManager(RoutedAgent):
    def __init__(
        self,
        participant_topic_types: List[str],
        model_client: ChatCompletionClient,
        participant_descriptions: List[str],
    ) -> None:
        super().__init__("Group chat manager")
        self._participant_topic_types = participant_topic_types
        self._model_client = model_client
        self._chat_history: List[UserMessage] = []
        self._participant_descriptions = participant_descriptions
        self._previous_participant_topic_type: str | None = None

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        assert isinstance(message.body, UserMessage)
        self._chat_history.append(message.body)
        # If the message is an approval message from the user, stop the chat.
        if message.body.source == "User":
            assert isinstance(message.body.content, str)
            if message.body.content.lower().strip(string.punctuation).endswith("approve"):
                return
        # Format message history.
        messages: List[str] = []
        for msg in self._chat_history:
            if isinstance(msg.content, str):
                messages.append(f"{msg.source}: {msg.content}")
            elif isinstance(msg.content, list):
                line: List[str] = []
                for item in msg.content:
                    if isinstance(item, str):
                        line.append(item)
                    else:
                        line.append("[Image]")
                messages.append(f"{msg.source}: {', '.join(line)}")
        history = "\n".join(messages)
        # Format roles.
        roles = "\n".join(
            [
                f"{topic_type}: {description}".strip()
                for topic_type, description in zip(
                    self._participant_topic_types, self._participant_descriptions, strict=True
                )
                if topic_type != self._previous_participant_topic_type
            ]
        )
        selector_prompt = """You are in a role play game. The following roles are available:
{roles}.
Read the following conversation. Then select the next role from {participants} to play. Only return the role.

{history}

Read the above conversation. Then select the next role from {participants} to play. Only return the role.
"""
        system_message = SystemMessage(
            content=selector_prompt.format(
                roles=roles,
                history=history,
                participants=str(
                    [
                        topic_type
                        for topic_type in self._participant_topic_types
                        if topic_type != self._previous_participant_topic_type
                    ]
                ),
            )
        )
        completion = await self._model_client.create([system_message], cancellation_token=ctx.cancellation_token)
        assert isinstance(completion.content, str)
        selected_topic_type: str
        for topic_type in self._participant_topic_types:
            if topic_type.lower() in completion.content.lower():
                selected_topic_type = topic_type
                self._previous_participant_topic_type = selected_topic_type
                await self.publish_message(RequestToSpeak(), DefaultTopicId(type=selected_topic_type))
                return
        raise ValueError(f"Invalid role selected: {completion.content}")


In [25]:
runtime = SingleThreadedAgentRuntime()

editor_topic_type = "Editor"
writer_topic_type = "Writer"
illustrator_topic_type = "Illustrator"
user_topic_type = "User"
group_chat_topic_type = "group_chat"

editor_description = "Editor for planning and reviewing the content."
writer_description = "Writer for creating any text content."
user_description = "User for providing final approval."
illustrator_description = "An illustrator for creating images."

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)

editor_agent_type = await EditorAgent.register(
    runtime,
    editor_topic_type,  # Using topic type as the agent type.
    lambda: EditorAgent(
        description=editor_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(TypeSubscription(topic_type=editor_topic_type, agent_type=editor_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=editor_agent_type.type))

writer_agent_type = await WriterAgent.register(
    runtime,
    writer_topic_type,  # Using topic type as the agent type.
    lambda: WriterAgent(
        description=writer_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(TypeSubscription(topic_type=writer_topic_type, agent_type=writer_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=writer_agent_type.type))

illustrator_agent_type = await IllustratorAgent.register(
    runtime,
    illustrator_topic_type,
    lambda: IllustratorAgent(
        description=illustrator_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
        image_client=openai.AsyncClient(),   
        ),
)
await runtime.add_subscription(
    TypeSubscription(topic_type=illustrator_topic_type, agent_type=illustrator_agent_type.type)
)
await runtime.add_subscription(
    TypeSubscription(topic_type=group_chat_topic_type, agent_type=illustrator_agent_type.type)
)

user_agent_type = await UserAgent.register(
    runtime,
    user_topic_type,
    lambda: UserAgent(description=user_description, group_chat_topic_type=group_chat_topic_type),
)
await runtime.add_subscription(TypeSubscription(topic_type=user_topic_type, agent_type=user_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=user_agent_type.type))

group_chat_manager_type = await GroupChatManager.register(
    runtime,
    "group_chat_manager",
    lambda: GroupChatManager(
        participant_topic_types=[writer_topic_type, illustrator_topic_type, editor_topic_type, user_topic_type],
        model_client=model_client,
        participant_descriptions=[writer_description, illustrator_description, editor_description, user_description],
    ),
)
await runtime.add_subscription(
    TypeSubscription(topic_type=group_chat_topic_type, agent_type=group_chat_manager_type.type)
)


In [24]:
runtime.start()
session_id = str(uuid.uuid4())
await runtime.publish_message(
    GroupChatMessage(
        body=UserMessage(
            content="Please write a short story about the gingerbread man with up to 3 photo-realistic illustrations.",
            source="User",
        )
    ),
    TopicId(type=group_chat_topic_type, source=session_id),
)
await runtime.stop_when_idle()
await model_client.close()


### User

### Writer

#### Writer Response

**Title: The Great Escape of the Gingerbread Man**

In a quaint little village, nestled between rolling hills and thick woods, there lay a vibrant bakery known for its delectable pastries and enchanting confections. The aroma of freshly baked goods wafted through the cobblestone streets, drawing villagers and travelers alike. Among the baker's most celebrated creations was a particularly spirited gingerbread man.

**Illustration 1: A bustling village bakery**
The scene captures the warm, inviting interior of the bakery. Stacks of golden-brown pastries and glistening tarts line the wooden counters. In the foreground, a tray of gingerbread men waits to be purchased, each adorned with icing and sugared buttons. Sunlight filters through the window, casting a gentle glow over the baked goods.

One crisp morning, as the baker fashioned yet another batch of gingerbread men, something magical stirred in the air. The last gingerbread man, with raisins for eyes and a mischievous grin piped lovingly onto his face, blinked. Snapping into action, he leapt from the baking tray, landing with a soft thud on the flour-dusted countertop.

"Whoa!" exclaimed the baker, taken aback.

But the gingerbread man, with a swift and cheeky wave, dashed across the bakery. "Run, run, as fast as you can! You can't catch me, I'm the gingerbread man!" he taunted with a merry laugh.

And off he went, out the door and into the bustling village streets.

**Illustration 2: The gingerbread man's flight through the village**
This image shows the gingerbread man sprinting alongside a cobblestone path, the vibrant village scenery blurring in the background. Villagers, caught by surprise, look on in astonishment and amusement. The lively colors and energetic composition capture the momentum and whimsy of the chase.

The gingerbread man sped past the marketplace, dodging between wooden carts and stalls laden with fresh produce. Villagers and their children joined the playful pursuit, spurred by his infectious cheer. Field mice darted from his path, while a stray cat paused its grooming to watch the spectacle.

The cheeky confection neared the edge of the village, his pace light and swift. As he neared the embankment of a babbling brook, a determined fox looked up from its watery reflection. The fox's ears perked at the whiff of gingerbread in the air.

"Where are you off to in such a hurry?" the fox called out with a sly glint in its eye.

With a nimble leap, the gingerbread man landed on the fox's nose, pausing just long enough to declare, "I run from the baker, the village folk, too. Try as you might, you can't catch me!"

The fox, smart and shrewd, feigned innocence before gently flicking the gingerbread man off his nose. "It’s not about catching, my little friend. It's about the journey," he said with a grin.

The gingerbread man laughed, a sound like tinkling bells, as he scampered back down the path. The fox watched, amused, before shrugging and dipping its paws back into the brook.

**Illustration 3: The gingerbread man united with nature**
This final illustration portrays the gingerbread man standing atop a hill, overlooking the village as dusk sets in. His form silhouetted against a sky painted in hues of pink and orange, he gazes out, triumphant and free. The surrounding meadow, filled with wildflowers and whispers of the evening breeze, extends peacefully into the distance.

As twilight descended upon the village, the gingerbread man found solace in the embrace of the countryside, his existence now a blink in the tapestry of the earth’s rhythm. And somewhere, between the notes of crickets' song and the rustling leaves, echoed his joyous refrain—forever running, forever free.

### User

...

### Editor

#### Editor Response

Thank you for your response. It seems that there might be some misunderstanding or additional direction you would like us to take. Could you please clarify what specific areas you'd like us to address or modify further? Whether it's about the narrative, character portrayal, illustrations, or another aspect of the draft or plan, I’m here to assist and ensure the final product meets your expectations. Your feedback is important to make the story and illustrations just right. Let me know how we can proceed!

### User

### User